In [1]:
import pandas as pd
import numpy as np
import gc
import os
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import joblib 

df = pd.read_feather('1차_test.feather')
print("로딩 완료.")

# --- 3. 시간 관련 특성 생성 ---
print("U/V 벡터에서 wind_spd (풍속) 값을 역산합니다...")
if 'U_a' in df.columns and 'V_a' in df.columns:
    df['wind_spd_a'] = np.sqrt(df['U_a']**2 + df['V_a']**2)
if 'U_b' in df.columns and 'V_b' in df.columns:
    df['wind_spd_b'] = np.sqrt(df['U_b']**2 + df['V_b']**2)
print("풍속 역산 완료.")

print("시간 관련 특성을 생성합니다...")
df['minute'] = df['time'].dt.minute
df['hour'] = df['time'].dt.hour
df['decimal_hour'] = df['hour'] + (df['minute'] / 60.0)
df['day_of_year'] = df['time'].dt.dayofyear
df['hour_sin'] = np.sin(2 * np.pi * df['decimal_hour'] / 24.0)
df['hour_cos'] = np.cos(2 * np.pi * df['decimal_hour'] / 24.0)
df['day_of_year_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365.25)
df['day_of_year_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365.25)
df['seasonal_hour'] = df['day_of_year_sin'] * df['hour_sin']

# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
# [수정된 섹션 3] 2-Cluster 전략 (Location + Climate) 적용
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★

print("위치 좌표(coord)와 '평균 기후'를 기반으로 2개의 '스마트 클러스터'를 적용합니다...")

# 3-1. 프로필 테이블 생성 (Test 데이터 기준, nins 없음)
print("각 발전소의 1년 평균 기후(temp, humidity)를 계산합니다...")
pv_id_stats = df.groupby('pv_id')[['temp_a', 'humidity', 'wind_spd_a', 'pressure', 'vis']].mean().reset_index()
pv_id_stats.rename(columns={
    'temp_a': 'avg_temp',
    'humidity': 'avg_humidity',
    'wind_spd_a': 'avg_wind',
    'pressure': 'avg_pressure',
    'vis': 'avg_vis'
}, inplace=True)

# 3-2. KMeans 재료 테이블 생성
unique_locations = df[['pv_id', 'coord1', 'coord2']].drop_duplicates(subset=['pv_id'])
unique_locations = pd.merge(unique_locations, pv_id_stats, on='pv_id', how='left')

# 3-3. ★ (수정) ★ 2개의 특성 리스트 정의 (Train과 동일)
cluster_features_LOC = ['coord1', 'coord2'] # (지리 군집용)

cluster_features_CLIMATE = [ # (기후 군집용)
    'avg_temp', 'avg_humidity',
    'avg_wind', 'avg_pressure', 'avg_vis'
]
all_cluster_features = cluster_features_LOC + cluster_features_CLIMATE

# 3-4. 결측치 처리 (fillna는 if/else 밖에서 실행)
nan_check = unique_locations[all_cluster_features].isnull()
if nan_check.any().any():
    print("\n!! 경고: Test KMeans 재료에서 결측치가 발견되었습니다!!")
    nan_counts_per_column = nan_check.sum()
    print(nan_counts_per_column[nan_counts_per_column > 0])
else:
    print("Test KMeans 군집 재료에 결측치가 없는 것을 확인했습니다.")
    
unique_locations[all_cluster_features] = unique_locations[all_cluster_features].fillna(0)
print("결측치 .fillna(0) 처리 완료.")


# 3-5. ★ (수정) ★ Cluster 1: Location Cluster (지리 군집) 적용
print("저장된 (scaler_loc)와 (kmeans_loc) 모델을 불러옵니다...")
scaler_loc = joblib.load('kmeans_scaler_loc.joblib')
kmeans_loc = joblib.load('kmeans_model_loc.joblib')

print("'location_cluster' (지리 군집)을 예측합니다...")
scaled_features_loc = scaler_loc.transform(unique_locations[cluster_features_LOC])
kmeans_loc.cluster_centers_ = kmeans_loc.cluster_centers_.astype(np.float64)
scaled_features_loc_64 = scaled_features_loc.astype(np.float64)
unique_locations['location_cluster'] = kmeans_loc.predict(scaled_features_loc_64)


# 3-6. ★ (수정) ★ Cluster 2: Climate Cluster (기후 군집) 적용
print("저장된 (scaler_climate)와 (kmeans_climate) 모델을 불러옵니다...")
scaler_climate = joblib.load('kmeans_scaler_climate.joblib')
kmeans_climate = joblib.load('kmeans_model_climate.joblib')

print("'climate_cluster' (기후 군집)을 예측합니다...")
scaled_features_climate = scaler_climate.transform(unique_locations[cluster_features_CLIMATE])
kmeans_climate.cluster_centers_ = kmeans_climate.cluster_centers_.astype(np.float64)
scaled_features_climate_64 = scaled_features_climate.astype(np.float64)
unique_locations['climate_cluster'] = kmeans_climate.predict(scaled_features_climate_64)


# 3-7. ★ (수정) ★ 2개의 '맵' 적용
print("메모리 효율적인 'cluster_map' 2종을 생성합니다...")
cluster_map_LOC = unique_locations.set_index('pv_id')['location_cluster']
cluster_map_CLIMATE = unique_locations.set_index('pv_id')['climate_cluster']

print(".map()을 사용하여 'location_cluster'와 'climate_cluster' 열을 추가합니다...")
df['location_cluster'] = df['pv_id'].map(cluster_map_LOC)
df['climate_cluster'] = df['pv_id'].map(cluster_map_CLIMATE) # (신규)

df['location_cluster'] = df['location_cluster'].astype('category')
df['climate_cluster'] = df['climate_cluster'].astype('category') # (신규)
print("위치/기후 군집 매핑 완료.")

# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
# [수정된 섹션 4] '맵' 파일 2종 적용 (각각 다른 군집 기준)


print("클러스터별 '개인화 일출/일몰 맵'을 불러옵니다 (location_cluster 기준)...")
try:
    cluster_times = pd.read_csv('cluster_sun_times_map.csv')
    sunrise_map = cluster_times.set_index('location_cluster')['sunrise_decimal']
    sunset_map = cluster_times.set_index('location_cluster')['sunset_decimal']
    
    # ★ (수정) ★ 'location_cluster' (지리 군집) 기준으로 맵핑
    df['sunrise_decimal'] = df['location_cluster'].map(sunrise_map)
    df['sunset_decimal'] = df['location_cluster'].map(sunset_map)
    print("일출/일몰 맵 병합 완료.")
except FileNotFoundError:
    print("경고: 'cluster_sun_times_map.csv' 파일을 찾을 수 없습니다.")
    df['sunrise_decimal'] = 0
    df['sunset_decimal'] = 0

print("GroupBy 개인화 기준선 맵 불러오는 중 (climate_cluster 기준)...")
try:
    base_df = pd.read_csv('cluster_hour_means_map.csv')
    
    # ★ (수정) ★ 'climate_cluster' (기후 군집) 기준으로 병합
    df = pd.merge(df, base_df, on=['climate_cluster', 'hour'], how='left')
    print("개인화 기준선 맵 병합 완료.")
except FileNotFoundError:
    print("경고: 'cluster_hour_means_map.csv' 파일을 찾을 수 없습니다.")
    # (Anomaly 기준열 7개 0으로 채우기)
    df['cluster_hour_cloud_mean'] = 0
    df['cluster_hour_uv_mean'] = 0
    df['cluster_hour_temp_mean'] = 0
    df['cluster_hour_humidity_mean'] = 0
    df['cluster_hour_vis_mean'] = 0
    df['cluster_hour_wind_mean'] = 0
    df['cluster_hour_pressure_mean'] = 0

# ★★★ (필수) ★★★ Rolling/Lag 계산 전, 시간 순서로 재정렬
print("Rolling/Lag 특성 생성을 위해 DataFrame을 재정렬합니다...")
df.sort_values(by=['pv_id', 'time'], inplace=True)

print("'평소 대비 이상 기후' (Anomaly) 특징 생성 중...")
# (Anomaly 계산 로직은 동일)
epsilon = 1e-6
df['cloud_anomaly'] = df['cloud_a'] / (df['cluster_hour_cloud_mean'] + epsilon)
df['uv_anomaly'] = df['uv_idx'] / (df['cluster_hour_uv_mean'] + epsilon)
df['temp_anomaly'] = df['temp_a'] / (df['cluster_hour_temp_mean'] + epsilon)
df['humidity_anomaly'] = df['humidity'] / (df['cluster_hour_humidity_mean'] +epsilon) 
df['vis_anomaly'] = df['vis'] / (df['cluster_hour_vis_mean']+epsilon)
df['wind_anomaly'] = df['wind_spd_a'] / (df['cluster_hour_wind_mean']+epsilon)
df['pressure_anomaly'] = df['pressure'] / (df['cluster_hour_pressure_mean']+epsilon)


# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
# [수정된 섹션 5] (섹션 5는 수정할 내용이 없습니다)
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★

print("이론적 태양광 강도 (Daylight Cosine) 특성을 생성합니다...")
# (수정 없음) 'sunrise_decimal' 값은 이미 'location_cluster' 기준으로 올바르게 매핑됨
df['sunrise_decimal'] = df['sunrise_decimal'].astype(np.float32).fillna(4.5)
df['sunset_decimal'] = df['sunset_decimal'].astype(np.float32).fillna(20.5)

daylight_duration = df['sunset_decimal'] - df['sunrise_decimal']
solar_noon = df['sunrise_decimal'] + (daylight_duration / 2.0)
scaled_time = (df['decimal_hour'] - solar_noon) * np.pi / daylight_duration
df['daylight_cosine'] = np.cos(scaled_time)

df.loc[
    (df['decimal_hour'] < df['sunrise_decimal']) | 
    (df['decimal_hour'] > df['sunset_decimal']), 
    'daylight_cosine'
] = 0
df.loc[df['daylight_cosine'] < 0, 'daylight_cosine'] = 0
print("Daylight Cosine 특성 생성 완료.")

# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
# [수정/교체] --- 날씨 '추세' 특성 생성 (15분, 30분, 1시간) ---
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
print("날씨 추세(Rolling/Lag/Std) 특성 생성 중...")

# (sort_values가 이미 위에서 실행되었으므로 여기서는 g = df.groupby('pv_id')만 실행)
g = df.groupby('pv_id')

# 1. 적용할 주요 날씨 특성 정의
weather_cols = ['cloud_a', 'temp_a', 'uv_idx', 'pressure', 'humidity']

# 2. 적용할 시간 윈도우 정의 (5분 단위)
windows = [3, 6, 12] # (15분, 30분, 1시간)

print(f" {len(weather_cols)}개 변수에 대해 {windows} 윈도우 Lag/Rolling/Std 특성을 생성합니다.")

for col in weather_cols:
    for w in windows:
        df[f'{col}_lag_{w}'] = g[col].shift(w)
        df[f'{col}_rolling_{w}_mean'] = g[col].shift(1).rolling(w, min_periods=1).mean()
        df[f'{col}_rolling_{w}_std'] = g[col].shift(1).rolling(w, min_periods=1).std()
        df[f'{col}_diff_{w}'] = g[col].diff(w)
        df[f'{col}_ewm_{w}_mean'] = g[col].shift(1).ewm(span=w, min_periods=1).mean()

print("다양한 시간대의 Rolling/Lag/Std 특성 생성 완료.")

# --- 기타 차이 및 물리 기반 특성 생성 ---
print("기타 차이 및 물리 기반 특성 생성 중...")
df['is_precipitating'] = ((df['precip_1h'] > 0) | (df['rain'] > 0) | (df['snow'] > 0)).astype(int)
df['station_temp_diff'] = df['temp_b'] - df['temp_a']
df['station_cloud_diff'] = df['cloud_a'] - df['cloud_b']
df['station_wind_spd_diff'] = df['wind_spd_a'] - df['wind_spd_b']

print("물리/풍향/상호작용 특성 (is_foggy, wind_power, _x_hour 등)을 생성합니다...")
df['temp_a_sq'] = df['temp_a']**2
df['uv_idx_sq'] = df['uv_idx']**2
df['sun_exposure_factor'] = df['real_feel_temp'].values - df['real_feel_temp_shade'].values
df['diurnal_temp_range'] = df['temp_max'].values - df['temp_min'].values
df['saturation_deficit'] = df['temp_a'].values - df['dew_point'].values 
df['is_foggy'] = (df['vis'] < 1).astype(int)
df['wind_power_a'] = df['wind_spd_a'] ** 3
df['wind_power_b'] = df['wind_spd_b'] ** 3
df['temp_x_hour'] = df['temp_a'] * df['hour_cos']
df['cloud_x_hour'] = df['cloud_a'] * df['hour_cos']
df['uv_idx_x_hour'] = df['uv_idx'] * df['hour_cos']
df['humidity_x_hour'] = df['humidity'] * df['hour_cos']
gc.collect()

# --- 최종 결측치 처리 (fillna) ---
print("최종 결측치 처리 (fillna) 중...")
lag_rolling_cols = [col for col in df.columns if '_lag_' in col or '_rolling_' in col]
df[lag_rolling_cols] = df[lag_rolling_cols].fillna(-999)

if 'is_precipitating' in df.columns:
     df['is_precipitating'] = df['is_precipitating'].fillna(-999)
if 'is_foggy' in df.columns: 
     df['is_foggy'] = df['is_foggy'].fillna(-999)

known_non_numeric_cols = ['time', 'pv_id', 'location_cluster', 'climate_cluster'] # (climate_cluster 추가)
cols_to_fill = [col for col in df.columns if col not in known_non_numeric_cols]

df[cols_to_fill] = df[cols_to_fill].fillna(-999)

# (location_cluster와 climate_cluster의 NaN을 -999 카테고리로 채움)
for col in ['location_cluster', 'climate_cluster']:
    if df[col].isnull().any():
        print(f"'{col}'의 NaN을 '-999' 카테고리로 채웁니다.")
        if -999 not in df[col].cat.categories:
             df[col] = df[col].cat.add_categories([-999])
        df[col] = df[col].fillna(-999)

gc.collect()

# --- 메모리 최적화 ---
print("모든 float64 칼럼을 float32로 강제 변환하여 메모리 사용량을 절반으로 줄입니다...")
numeric_cols = [col for col in df.columns if col not in known_non_numeric_cols]
float64_cols = [col for col in numeric_cols if df[col].dtype == 'float64']
for col in float64_cols:
    df[col] = df[col].astype('float32')
print(f"{len(float64_cols)}개 칼럼을 float32로 변환 완료.")
gc.collect()

# --- 저장 ---
print("중간 처리된 데이터를 'processed_test.feather' 파일로 저장합니다...")
df.to_feather('processed_test.feather')
print("저장 완료.")

KeyboardInterrupt: 